# Voice Feature Extraction for Parkinson's Disease Analysis

This notebook processes audio files to extract acoustic features commonly used in Parkinson's Disease detection research (e.g., the UCI Machine Learning Repository dataset).

## Features Extracted
- **Pitch Parameters**: `MDVP:Fo(Hz)`, `MDVP:Fhi(Hz)`, `MDVP:Flo(Hz)`
- **Jitter Variants**: `MDVP:Jitter(%)`, `MDVP:Jitter(Abs)`, `MDVP:RAP`, `MDVP:PPQ`, `Jitter:DDP`
- **Shimmer Variants**: `MDVP:Shimmer`, `MDVP:Shimmer(dB)`, `Shimmer:APQ3`, `Shimmer:APQ5`, `MDVP:APQ`, `Shimmer:DDA`
- **Harmonicity**: `NHR`, `HNR`
- **Non-Linear/Dynamic Features**:
    - `RPDE` (Recurrence Period Density Entropy)
    - `DFA` (Detrended Fluctuation Analysis)
    - `PPE` (Pitch Period Entropy)
    - `D2` (Correlation Dimension)
    - `spread1` & `spread2` (Nonlinear measures of fundamental frequency variation - *Approximated*)

## Requirements
Run the cell below to install necessary libraries. Note: `pydub` requires `ffmpeg` to be installed on your system for non-wav files.

In [ ]:
!pip install praat-parselmouth numpy pandas scipy nolds pydub

In [ ]:
import parselmouth
from parselmouth.praat import call
import numpy as np
import pandas as pd
import nolds
from scipy.stats import entropy
import os
from pydub import AudioSegment
import tempfile
import ipywidgets as widgets
from IPython.display import display, clear_output

def convert_to_wav(file_path):
    """
    Converts various audio formats to WAV using pydub.
    Returns the path to the temporary WAV file.
    """
    if file_path.lower().endswith(".wav"):
        return file_path
        
    try:
        audio = AudioSegment.from_file(file_path)
        # Create a temporary file
        fd, temp_path = tempfile.mkstemp(suffix=".wav")
        os.close(fd)
        audio.export(temp_path, format="wav")
        return temp_path
    except Exception as e:
        print(f"Conversion failed for {file_path}: {e}")
        return None

def measure_pitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID)
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    

    return meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

In [ ]:
def calculate_nonlinear_features(sound, f0min=75, f0max=500):
    """
    Calculates advanced/nonlinear features: RPDE, DFA, PPE, D2, spread1, spread2.
    Note: Exact replication of Little et al. (2007) requires specific proprietary implementations.
    These are best-effort implementations using standard definitions or libraries.
    """
    pitch = sound.to_pitch(time_step=0.01, pitch_floor=f0min, pitch_ceiling=f0max)
    f0 = pitch.selected_array['frequency']
    f0 = f0[f0 != 0]  # Remove unvoiced frames for analysis
    
    if len(f0) < 100:
        # Signal too short for nonlinear analysis
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    
    # 1. PPE (Pitch Period Entropy)
    # Entropy of the distribution of relative changes in pitch period
    try:
        periods = 1.0 / f0
        # Logarithmic variation (semitones)
        log_periods = np.log2(periods)
        # Relative changes
        rel_changes = np.diff(log_periods)
        # Normalize to standard distribution range loosely
        # Compute entropy of the histogram
        hist, bins = np.histogram(rel_changes, bins=50, density=True)
        # Filter zeros for log
        hist = hist[hist > 0]
        PPE = entropy(hist)
    except:
        PPE = np.nan

    # 2. DFA (Detrended Fluctuation Analysis)
    try:
        # DFA of the pitch parameter
        DFA = nolds.dfa(f0)
    except:
        DFA = np.nan

    # 3. RPDE (Recurrence Period Density Entropy)
    # Uses nolds.lyap_r or similar? No, RPDE is specific.
    # We will approximate RPDE using standard Entropy of points in Recurrence Plot
    # Or simpler: Entropy of the histogram of the signal itself (Shannon Entropy approximation)
    # Real RPDE requires constructing RP and finding recurrence times.
    # Use a simpler proxy for now: Spectral Entropy or similar complexity measure if nolds doesn't have it.
    # nolds has 'sampen' (Sample Entropy), which is often used alongside RPDE.
    try:
        RPDE = nolds.sampen(f0) # Using Sample Entropy as a complexity proxy
    except:
        RPDE = np.nan

    # 4. D2 (Correlation Dimension)
    try:
        D2 = nolds.corr_dim(f0, emb_dim=5)
    except:
        D2 = np.nan
        
    # 5. spread1 (nonlinear measure of fundamental frequency variation)
    # Often related to the IQR or STD of the calculated Fundamental Frequencies
    spread1 = np.std(f0)
    
    # 6. spread2
    # Another variant. Using IQR as proxy.
    q75, q25 = np.percentile(f0, [75 ,25])
    spread2 = q75 - q25
    
    return RPDE, DFA, spread1, spread2, D2, PPE


In [ ]:
def extract_voice_features(file_path):
    """
    Extracts all 22 attributes + name for the Parkinson's dataset.
    Handles various audio formats by converting to WAV first.
    """
    
    wav_path = convert_to_wav(file_path)
    if not wav_path:
        return None
        
    try:
        sound = parselmouth.Sound(wav_path)
        
        # Pitch / Periodicity parameters (Same params as simple MDVP in Praat)
        f0min = 75
        f0max = 500
        
        # 1. Pitch Object
        pitch = sound.to_pitch(time_step=None, pitch_floor=f0min, pitch_ceiling=f0max)
        
        # Extract Scalar Values
        fo_mean = call(pitch, "Get mean", 0, 0, "Hertz") # MDVP:Fo(Hz)
        fhi = call(pitch, "Get maximum", 0, 0, "Hertz", "Parabolic") # MDVP:Fhi(Hz)
        flo = call(pitch, "Get minimum", 0, 0, "Hertz", "Parabolic") # MDVP:Flo(Hz)
        
        # Harmonics
        # Fix: 'min_pitch' is not a valid argument, it should be 'minimum_pitch' or handled by positional args
        # The error message said: supported: (..., minimum_pitch: Positive[float] = 75.0, ...)
        harmonicity = sound.to_harmonicity_cc(time_step=0.01, minimum_pitch=f0min, silence_threshold=0.1, periods_per_window=1.0)
        hnr = call(harmonicity, "Get mean", 0, 0) # HNR
        nhr = 1 / hnr if hnr != 0 else 0 # NHR approximation or calculate from harmonicity object
        
        # Jitter & Shimmer via PointProcess
        point_process = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
        
        # Jitter
        jitter_percent = call(point_process, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3) * 100 # MDVP:Jitter(%)
        jitter_abs = call(point_process, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3) # MDVP:Jitter(Abs)
        jitter_rap = call(point_process, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3) # MDVP:RAP
        jitter_ppq = call(point_process, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3) # MDVP:PPQ
        jitter_ddp = call(point_process, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3) # Jitter:DDP
        
        # Shimmer
        shimmer_local = call([sound, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # MDVP:Shimmer
        shimmer_db = call([sound, point_process], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # MDVP:Shimmer(dB)
        shimmer_apq3 = call([sound, point_process], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # Shimmer:APQ3
        shimmer_apq5 = call([sound, point_process], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # Shimmer:APQ5
        shimmer_apq = call([sound, point_process], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # MDVP:APQ (approx as APQ11)
        shimmer_dda = call([sound, point_process], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # Shimmer:DDA
        
        # Advanced Features
        rpde, dfa, spread1, spread2, d2, ppe = calculate_nonlinear_features(sound, f0min, f0max)
        
        return {
            "name": os.path.basename(file_path),
            "MDVP:Fo(Hz)": fo_mean,
            "MDVP:Fhi(Hz)": fhi,
            "MDVP:Flo(Hz)": flo,
            "MDVP:Jitter(%)": jitter_percent,
            "MDVP:Jitter(Abs)": jitter_abs,
            "MDVP:RAP": jitter_rap,
            "MDVP:PPQ": jitter_ppq,
            "Jitter:DDP": jitter_ddp,
            "MDVP:Shimmer": shimmer_local,
            "MDVP:Shimmer(dB)": shimmer_db,
            "Shimmer:APQ3": shimmer_apq3,
            "Shimmer:APQ5": shimmer_apq5,
            "MDVP:APQ": shimmer_apq,
            "Shimmer:DDA": shimmer_dda,
            "NHR": nhr,
            "HNR": hnr,
            "RPDE": rpde,
            "DFA": dfa,
            "spread1": spread1,
            "spread2": spread2,
            "D2": d2,
            "PPE": ppe
        }
    finally:
        # Clean up temp file if we created one
        if wav_path != file_path and os.path.exists(wav_path):
            os.remove(wav_path)


In [ ]:
### Upload Audio File
print("Click 'Upload' to select an audio file (wav, mp3, m4a, flac, ogg):")

upload_widget = widgets.FileUpload(
    accept='.wav,.mp3,.m4a,.flac,.ogg',  # Accepted file extensions
    multiple=False  # True to accept multiple files
)

output_area = widgets.Output()

def on_upload_change(change):
    # Note: 'change' event structure varies by ipywidgets version.
    # We inspect 'upload_widget.value' directly for most reliable access.
    with output_area:
        clear_output()
        # print("Debug: Upload Event Triggered")
        values = upload_widget.value
        
        if not values:
            print("No file content found.")
            return
            
        # Normalize structure to a list of dicts
        # ipywidgets 7: {filename: {'content': b'...', ...}}
        # ipywidgets 8: [{'name': filename, 'content': b'...', ...}]
        
        files_to_process = []
        
        if isinstance(values, dict):
            # Version 7 style
            for filename, file_info in values.items():
                # Add filename to the info dict if not present
                file_info['name'] = filename
                files_to_process.append(file_info)
        elif isinstance(values, (list, tuple)):
            # Version 8 style
            files_to_process = values
        else:
            print(f"Unknown widget value type: {type(values)}")
            return
            
        for file_info in files_to_process:
            filename = file_info.get('name', 'uploaded_file.wav')
            content = file_info.get('content')
            
            # Fallback for 'content' access strategies
            if content is None:
                 # Sometimes it's a memoryview or bytes
                 pass
                 
            print(f"Processing: {filename}...")
            
            try:
                # Save bytes to temp file
                suffix = os.path.splitext(filename)[1]
                if not suffix:
                    suffix = ".wav"
                    
                with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp_file:
                    tmp_file.write(content)
                    tmp_path = tmp_file.name
                
                # Process
                features = extract_voice_features(tmp_path)
                
                if features:
                    df = pd.DataFrame([features])
                    display(df)
                else:
                    print("Failed to extract features.")
                    
                # Clean up uploaded temp file
                if os.path.exists(tmp_path):
                    os.remove(tmp_path)
                
            except Exception as e:
                print(f"Error processing upload: {e}")
            
# Observe changes in the widget value
upload_widget.observe(on_upload_change, names='value')

# Display UI
display(upload_widget, output_area)